In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

In [19]:
user_df = pd.read_csv('data/steam_new_users.csv')
user_ids = user_df.loc[:, ["user_id", "public"]].to_dict(orient='list')

In [3]:
def if_have_games(user_id):
    req = f'https://steamcommunity.com/profiles/{user_id}/games?tab=all&xml=1'
    # print(req)
    num_tries = 3
    while num_tries >= 0:
        resp = requests.get(req)
        if resp.status_code != 200:
            print(f"Status code: {resp.status_code}")
            num_tries -= 1 
        from xml.etree import ElementTree
        from xml.etree.ElementTree import ParseError
        try:
            xml_tree = ElementTree.fromstring(resp.content)
            break
        except ParseError:
            num_tries -= 1 
    
    if num_tries < 0:
        return False
    
    for elem in xml_tree.iter('*'):
        if elem.tag == "game":
            return True
    return False

In [15]:
API_KEY = 'F0F269DD995EA073497C8E71B10F61B7'
request = f'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={API_KEY}&steamid=<steamid>relationship=all'
users = len(user_ids)
num_of_requests = 0
while num_of_requests < 100:
    try:
        user = user_df[user_df['used'] == False].sample(1).index.values[0]
        response = requests.get(request.replace("<steamid>", str(user_ids['user_id'][user])))
        if response.status_code == 200:
            user_df.loc[user, 'used'] = True
            friends = response.json()
            for friend in friends['friendslist']['friends']:
                new_steam_id = friend['steamid']
                if new_steam_id not in user_ids['user_id']:
                    user_ids['user_id'].append(new_steam_id)
                    user_ids['public'].append(if_have_games(new_steam_id))
        num_of_requests += 1
        users = len(user_ids['user_id'])
        print(users)
    except IndexError:
        print("No users left")
        break
    except ConnectionError:
        num_of_requests += 1
        continue


21192
21218
21286
21300


KeyboardInterrupt: 

In [17]:
# users_series = pd.Series({"user_id": user_ids})
users_new_df = pd.DataFrame.from_dict(user_ids) # pd.DataFrame({"user_id": user_ids})
users_new_df['used'] = False
users_new_df.loc[(user_df[user_df['used'] == True]).index, 'used'] = True

In [18]:
users_new_df.to_csv('data/steam_new_users.csv', index=False)

In [79]:
user_df[user_df['used'] == False].sample(1).index.values[0]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [65]:
# req = 'https://steamcommunity.com/profiles/76561198148329339/games?tab=all&xml=1'

req = 'https://steamcommunity.com/profiles/76561198142848520/games?tab=all&xml=1'
resp = requests.get(req)

In [5]:
user_df.head()

,user_id,public,used
0,76561198036259748,False,False
1,76561197963906371,False,False
2,76561197997562600,False,False
3,76561198026410495,False,False
4,76561198028315881,False,False


In [18]:
user_ids = user_df.loc[:, ["user_id", "public"]].to_dict(orient='list')

In [20]:
# check users up to 5680 on privacy of accounts
users_new_df.shape

(21349, 3)

In [24]:
len(user_ids['user_id'])

6067

In [16]:
user_ids['user_id'] = user_ids['user_id'][:-1]

In [27]:
users_new_df[users_new_df['public'] == True].shape

(741, 3)